<a href="https://colab.research.google.com/github/ElizavetaNosova/HSE_ML_homework/blob/master/NeuralNerworksIntroHW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Я выбрала вторую задачу - классификацию токсичных вопросов

In [1]:
from google.colab import files
uploaded = files.upload()

Saving quora.csv to quora.csv


In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [0]:
data = pd.read_csv('quora.csv')

In [0]:
data

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0
...,...,...,...
1306117,ffffcc4e2331aaf1e41e,What other technical skills do you need as a c...,0
1306118,ffffd431801e5a2f4861,Does MS in ECE have good job prospects in USA ...,0
1306119,ffffd48fb36b63db010c,Is foam insulation toxic?,0
1306120,ffffec519fa37cf60c78,How can one start a research project based on ...,0


In [0]:
from string import punctuation
from collections import Counter 

def preprocess(text):
    tokens = text.lower().split()
    tokens = [token.strip(punctuation) for token in tokens]
    return tokens

In [0]:
vocab = Counter()

for text in data.question_text:
    vocab.update(preprocess(text))

In [0]:
filtered_vocab = set()

for word in vocab:
    if vocab[word] > 50:
        filtered_vocab.add(word)

In [0]:
len(filtered_vocab)

14083

In [0]:
import gensim

In [0]:
data['pr_text'] = data['question_text'].apply(preprocess)

In [0]:
data

,qid,question_text,target,pr_text
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0,"[how, did, quebec, nationalists, see, their, p..."
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0,"[do, you, have, an, adopted, dog, how, would, ..."
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0,"[why, does, velocity, affect, time, does, velo..."
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0,"[how, did, otto, von, guericke, used, the, mag..."
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0,"[can, i, convert, montra, helicon, d, to, a, m..."
...,...,...,...,...
1306117,ffffcc4e2331aaf1e41e,What other technical skills do you need as a c...,0,"[what, other, technical, skills, do, you, need..."
1306118,ffffd431801e5a2f4861,Does MS in ECE have good job prospects in USA ...,0,"[does, ms, in, ece, have, good, job, prospects..."
1306119,ffffd48fb36b63db010c,Is foam insulation toxic?,0,"[is, foam, insulation, toxic]"
1306120,ffffec519fa37cf60c78,How can one start a research project based on ...,0,"[how, can, one, start, a, research, project, b..."


In [0]:
w2v50 = gensim.models.Word2Vec(data['pr_text'], size=50, sg=1)

In [0]:
def sum_embeddings50(tokens):
  res = np.zeros(50)
  for token in tokens:
    if token in filtered_vocab:
        res += w2v50[token]
  return res

In [0]:
def mean_embeddings50(tokens):
  res = np.zeros(50)
  n = 0
  for token in tokens:
    if token in filtered_vocab:
        res += w2v50[token]
        n += 1
  if n:
      res = res * (1/n)
  #print('.') #работало безумо медленно, хотела посмотреть, работало ли вообще
  return res

In [0]:
data['sum50'] = data['pr_text'].apply(sum_embeddings50)

In [0]:
X_train, X_valid, y_train, y_valid = train_test_split(data['sum50'].values, data['target'].values, test_size=0.1, random_state=42)

In [0]:
def list_of_float_lists(npndarray):
  embedings = []
  for embeding in list(npndarray):
     new_embeding = [float(i) for i in list(embeding)]
     embedings.append(new_embeding)
  return embedings

In [0]:
X_train = list_of_float_lists(X_train)
X_valid = list_of_float_lists(X_valid)

In [0]:
y_train = [int(i) for i in list(y_train)]
y_valid = [int(i) for i in list(y_valid)]

In [0]:
inputs = tf.keras.layers.Input(shape=50)
dense1 = tf.keras.layers.Dense(64, activation='relu')(inputs)
dense2 = tf.keras.layers.Dense(64, activation='relu')(dense1)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(dense2)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [0]:
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=20000, epochs=10)

Epoch 1/10
59/59 [==============================] - 28s 477ms/step - loss: 0.4724 - accuracy: 0.8190 - val_loss: 0.3163 - val_accuracy: 0.9387
Epoch 2/10
59/59 [==============================] - 28s 472ms/step - loss: 0.3000 - accuracy: 0.9374 - val_loss: 0.2732 - val_accuracy: 0.9395
Epoch 3/10
59/59 [==============================] - 28s 474ms/step - loss: 0.2552 - accuracy: 0.9377 - val_loss: 0.2276 - val_accuracy: 0.9399
Epoch 4/10
59/59 [==============================] - 28s 473ms/step - loss: 0.2148 - accuracy: 0.9391 - val_loss: 0.1962 - val_accuracy: 0.9422
Epoch 5/10
59/59 [==============================] - 28s 472ms/step - loss: 0.1923 - accuracy: 0.9413 - val_loss: 0.1812 - val_accuracy: 0.9436
Epoch 6/10
59/59 [==============================] - 28s 472ms/step - loss: 0.1810 - accuracy: 0.9424 - val_loss: 0.1727 - val_accuracy: 0.9445
Epoch 7/10
59/59 [==============================] - 28s 476ms/step - loss: 0.1738 - accuracy: 0.9432 - val_loss: 0.1666 - val_accuracy: 0.9454

In [0]:
data['mean50'] = data['pr_text'].apply(mean_embeddings50)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


Попробуем усреднённые векторы, а не сумму векторов

In [0]:
X_train, X_valid, y_train, y_valid = train_test_split(data['mean50'].values, data['target'].values, test_size=0.1, random_state=42)
X_train = list_of_float_lists(X_train)
X_valid = list_of_float_lists(X_valid)
y_train = [int(i) for i in list(y_train)]
y_valid = [int(i) for i in list(y_valid)]

In [0]:
inputs = tf.keras.layers.Input(shape=50)
dense1 = tf.keras.layers.Dense(64, activation='relu')(inputs)
dense2 = tf.keras.layers.Dense(64, activation='relu')(dense1)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(dense2)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [0]:
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=20000, epochs=10)

Epoch 1/10
59/59 [==============================] - 26s 447ms/step - loss: 0.1701 - accuracy: 0.9389 - val_loss: 0.1621 - val_accuracy: 0.9409
Epoch 2/10
59/59 [==============================] - 26s 447ms/step - loss: 0.1632 - accuracy: 0.9400 - val_loss: 0.1564 - val_accuracy: 0.9423
Epoch 3/10
59/59 [==============================] - 26s 446ms/step - loss: 0.1583 - accuracy: 0.9408 - val_loss: 0.1521 - val_accuracy: 0.9428
Epoch 4/10
59/59 [==============================] - 27s 450ms/step - loss: 0.1547 - accuracy: 0.9414 - val_loss: 0.1490 - val_accuracy: 0.9432
Epoch 5/10
59/59 [==============================] - 27s 450ms/step - loss: 0.1521 - accuracy: 0.9418 - val_loss: 0.1467 - val_accuracy: 0.9438
Epoch 6/10
59/59 [==============================] - 26s 444ms/step - loss: 0.1500 - accuracy: 0.9422 - val_loss: 0.1448 - val_accuracy: 0.9440
Epoch 7/10
59/59 [==============================] - 26s 444ms/step - loss: 0.1484 - accuracy: 0.9425 - val_loss: 0.1433 - val_accuracy: 0.9442

Результаты использования суммы или среднего векторов различаются крайне несущественно (если брать только наиболее частотные слова).
В первом случае улучшение результатов между эпохами более ощутимое

Попробуем не обрезать словарь сверх настроек модели w2v

In [0]:
def sum_all_embeddings50(tokens):
  res = np.zeros(50)
  for token in tokens:
    try:
      res += w2v50[token]
    except KeyError:
      pass
  return res

In [13]:
data['sum_all50'] = data['pr_text'].apply(sum_all_embeddings50)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [0]:
X_train, X_valid, y_train, y_valid = train_test_split(data['sum_all50'].values, data['target'].values, test_size=0.1, random_state=42)
X_train = list_of_float_lists(X_train)
X_valid = list_of_float_lists(X_valid)
y_train = [int(i) for i in list(y_train)]
y_valid = [int(i) for i in list(y_valid)]

In [0]:
inputs = tf.keras.layers.Input(shape=50)
dense1 = tf.keras.layers.Dense(64, activation='relu')(inputs)
dense2 = tf.keras.layers.Dense(64, activation='relu')(dense1)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(dense2)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [0]:
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=20000, epochs=10)

Epoch 1/10
59/59 [==============================] - 26s 442ms/step - loss: 1.3047 - accuracy: 0.3116 - val_loss: 0.5487 - val_accuracy: 0.8024
Epoch 2/10
59/59 [==============================] - 26s 440ms/step - loss: 0.4369 - accuracy: 0.9021 - val_loss: 0.3740 - val_accuracy: 0.9358
Epoch 3/10
59/59 [==============================] - 26s 444ms/step - loss: 0.3554 - accuracy: 0.9368 - val_loss: 0.3282 - val_accuracy: 0.9395
Epoch 4/10
59/59 [==============================] - 26s 443ms/step - loss: 0.3144 - accuracy: 0.9379 - val_loss: 0.2882 - val_accuracy: 0.9398
Epoch 5/10
59/59 [==============================] - 26s 441ms/step - loss: 0.2728 - accuracy: 0.9379 - val_loss: 0.2461 - val_accuracy: 0.9398
Epoch 6/10
59/59 [==============================] - 26s 441ms/step - loss: 0.2289 - accuracy: 0.9381 - val_loss: 0.2022 - val_accuracy: 0.9410
Epoch 7/10
59/59 [==============================] - 26s 440ms/step - loss: 0.1900 - accuracy: 0.9412 - val_loss: 0.1754 - val_accuracy: 0.9450

In [0]:
def mean_all_embeddings50(tokens):
  res = np.zeros(50)
  n = 0
  for token in tokens:
    try:
        res += w2v50[token]
        n += 1
    except KeyError:
      pass
  if n:
      res = res * (1/n)
  return res

In [0]:
data['mean_all50'] = data['pr_text'].apply(mean_all_embeddings50)

In [0]:
X_train, X_valid, y_train, y_valid = train_test_split(data['mean_all50'].values, data['target'].values, test_size=0.1, random_state=42)
X_train = list_of_float_lists(X_train)
X_valid = list_of_float_lists(X_valid)
y_train = [int(i) for i in list(y_train)]
y_valid = [int(i) for i in list(y_valid)]

In [0]:
inputs = tf.keras.layers.Input(shape=50)
dense1 = tf.keras.layers.Dense(64, activation='relu')(inputs)
dense2 = tf.keras.layers.Dense(64, activation='relu')(dense1)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(dense2)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [0]:
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=20000, epochs=10)

Epoch 1/10
59/59 [==============================] - 26s 446ms/step - loss: 0.7922 - accuracy: 0.5981 - val_loss: 0.3275 - val_accuracy: 0.9164
Epoch 2/10
59/59 [==============================] - 26s 445ms/step - loss: 0.2950 - accuracy: 0.9310 - val_loss: 0.2691 - val_accuracy: 0.9382
Epoch 3/10
59/59 [==============================] - 26s 443ms/step - loss: 0.2585 - accuracy: 0.9374 - val_loss: 0.2371 - val_accuracy: 0.9400
Epoch 4/10
59/59 [==============================] - 26s 442ms/step - loss: 0.2277 - accuracy: 0.9389 - val_loss: 0.2098 - val_accuracy: 0.9410
Epoch 5/10
59/59 [==============================] - 26s 439ms/step - loss: 0.2050 - accuracy: 0.9401 - val_loss: 0.1921 - val_accuracy: 0.9421
Epoch 6/10
59/59 [==============================] - 26s 442ms/step - loss: 0.1905 - accuracy: 0.9414 - val_loss: 0.1806 - val_accuracy: 0.9434
Epoch 7/10
59/59 [==============================] - 26s 443ms/step - loss: 0.1806 - accuracy: 0.9423 - val_loss: 0.1726 - val_accuracy: 0.9442

Для суммы при использовании всех слов результат чуть-чуть улучшился, для среднего уменьшился на сотую долю процента

Добавим фасттекст

In [0]:
fast_text = gensim.models.FastText(data['pr_text'], size=50, min_n=4, max_n=8)

In [0]:
def fasttext_sum(tokens):
    embedings = []
    for token in tokens:
      try:
        embedings.append(fast_text[token])
      except KeyError:
        pass
    return sum(embedings)

In [16]:
data['fasttext_sum'] = data['pr_text'].apply(fasttext_sum)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [0]:
X = []
for index, row in data.iterrows():
    concat = []
    for i in list(row['sum_all50']):
      concat.append(float(i))
    for i in list(row['fasttext_sum']):
      concat.append(float(i))
    X.append(concat)
         

In [0]:
y = [int(i) for i in list(data['target'])]

In [0]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=42)

In [0]:
inputs = tf.keras.layers.Input(shape=100)
dense1 = tf.keras.layers.Dense(64, activation='relu')(inputs)
dense2 = tf.keras.layers.Dense(64, activation='relu')(dense1)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(dense2)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [21]:
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=20000, epochs=10)

Epoch 1/10
59/59 [==============================] - 51s 870ms/step - loss: 2.6067 - accuracy: 0.6158 - val_loss: 0.7169 - val_accuracy: 0.8837
Epoch 2/10
59/59 [==============================] - 51s 868ms/step - loss: 0.6264 - accuracy: 0.8912 - val_loss: 0.5396 - val_accuracy: 0.8925
Epoch 3/10
59/59 [==============================] - 51s 868ms/step - loss: 0.4895 - accuracy: 0.8938 - val_loss: 0.4405 - val_accuracy: 0.8981
Epoch 4/10
59/59 [==============================] - 51s 863ms/step - loss: 0.4149 - accuracy: 0.9016 - val_loss: 0.3847 - val_accuracy: 0.9063
Epoch 5/10
59/59 [==============================] - 51s 862ms/step - loss: 0.3709 - accuracy: 0.9083 - val_loss: 0.3499 - val_accuracy: 0.9131
Epoch 6/10
59/59 [==============================] - 53s 897ms/step - loss: 0.3419 - accuracy: 0.9134 - val_loss: 0.3261 - val_accuracy: 0.9172
Epoch 7/10
59/59 [==============================] - 51s 859ms/step - loss: 0.3210 - accuracy: 0.9170 - val_loss: 0.3079 - val_accuracy: 0.9206

Побробуем использовать векторы другой длины

In [0]:
w2v100 = gensim.models.Word2Vec(data['pr_text'], size=100, sg=1)

In [0]:
def sum_all_embeddings100(tokens):
  res = np.zeros(100)
  for token in tokens:
    try:
      res += w2v100[token]
    except KeyError:
      pass
  return res

In [9]:
data['w2v100'] = data['pr_text'].apply(sum_all_embeddings100)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [0]:
fast_text100 = gensim.models.FastText(data['pr_text'], size=100, min_n=4, max_n=8)

In [0]:
def fasttext_sum100(tokens):
    embedings = []
    for token in tokens:
      try:
        embedings.append(fast_text100[token])
      except KeyError:
        pass
    return sum(embedings)

In [13]:
data['fasttext100'] = data['pr_text'].apply(fasttext_sum100)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [0]:
X = []
for index, row in data.iterrows():
    concat = []
    for i in list(row['w2v100']):
      concat.append(float(i))
    for i in list(row['fasttext100']):
      concat.append(float(i))
    X.append(concat)
         

In [0]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=42)

In [0]:
inputs = tf.keras.layers.Input(shape=200)
dense1 = tf.keras.layers.Dense(64, activation='relu')(inputs)
dense2 = tf.keras.layers.Dense(64, activation='relu')(dense1)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(dense2)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [19]:
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=20000, epochs=10)

Epoch 1/10
59/59 [==============================] - 98s 2s/step - loss: 0.6480 - accuracy: 0.8741 - val_loss: 0.4324 - val_accuracy: 0.9045
Epoch 2/10
59/59 [==============================] - 96s 2s/step - loss: 0.3644 - accuracy: 0.9114 - val_loss: 0.3082 - val_accuracy: 0.9216
Epoch 3/10
59/59 [==============================] - 96s 2s/step - loss: 0.2882 - accuracy: 0.9229 - val_loss: 0.2620 - val_accuracy: 0.9277
Epoch 4/10
59/59 [==============================] - 97s 2s/step - loss: 0.2519 - accuracy: 0.9282 - val_loss: 0.2345 - val_accuracy: 0.9314
Epoch 5/10
59/59 [==============================] - 97s 2s/step - loss: 0.2292 - accuracy: 0.9312 - val_loss: 0.2163 - val_accuracy: 0.9337
Epoch 6/10
59/59 [==============================] - 97s 2s/step - loss: 0.2138 - accuracy: 0.9335 - val_loss: 0.2037 - val_accuracy: 0.9358
Epoch 7/10
59/59 [==============================] - 96s 2s/step - loss: 0.2027 - accuracy: 0.9352 - val_loss: 0.1941 - val_accuracy: 0.9373
Epoch 8/10
59/59 [==

При использовании векторов большей длины результат улучшился